# Exploration using Pandas

In [1]:
import pandas as pd
import os
import util

In [2]:
os.path.abspath('')

'/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/src'

In [2]:
base_url = 'https://datasets.imdbws.com'

files_list = [
        "name.basics.tsv.gz",
        "title.akas.tsv.gz",
        "title.basics.tsv.gz",
        "title.crew.tsv.gz",
        "title.episode.tsv.gz",
        "title.principals.tsv.gz",
        "title.ratings.tsv.gz"]
        
temp_filepath = os.path.abspath(os.path.join(os.path.abspath(''), os.pardir, 'data', 'tmp'))

In [4]:
util.download_files_to_local(base_url, files_list, temp_filepath)

In [20]:
data = dict()

for file in files_list:
    data[file] = pd.read_csv(temp_filepath+'/'+file, sep = '\t', nrows=100, compression='gzip',error_bad_lines=False)

In [21]:
data['name.basics.tsv.gz']['deathYear'] = data['name.basics.tsv.gz']['deathYear'].replace('\\N','') 
data['name.basics.tsv.gz'].head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0050419,tt0053137,tt0043044"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,,"actress,soundtrack,producer","tt0059956,tt0049189,tt0054452,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,writer,soundtrack","tt0077975,tt0072562,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050976,tt0083922,tt0050986"


In [24]:
data['title.akas.tsv.gz']['region'] = data['title.akas.tsv.gz']['region'].replace('\\N','XX') 
data['title.akas.tsv.gz'].head(20)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,XX,\N,original,\N,1
5,tt0000002,1,Le clown et ses chiens,XX,\N,original,\N,1
6,tt0000002,2,A bohóc és kutyái,HU,\N,imdbDisplay,\N,0
7,tt0000002,3,Le clown et ses chiens,FR,\N,\N,\N,0
8,tt0000002,4,Clovnul si cainii sai,RO,\N,imdbDisplay,\N,0
9,tt0000002,5,Клоун и его собаки,RU,\N,\N,\N,0


In [16]:
data["title.basics.tsv.gz"].head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [17]:
data["title.crew.tsv.gz"].head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [18]:
data["title.episode.tsv.gz"].head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


In [25]:
data["title.principals.tsv.gz"].head(20)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm5442194,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


In [20]:
data["title.ratings.tsv.gz"].head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1529
1,tt0000002,6.1,185
2,tt0000003,6.5,1175
3,tt0000004,6.2,113
4,tt0000005,6.1,1892


In [5]:
data["title.episode.tsv.gz"].describe()

,tconst,parentTconst,seasonNumber,episodeNumber
count,100,100,100,100
unique,100,30,10,30
top,tt0052290,tt0793361,1,\N
freq,1,13,36,10


# Exploration using PySpark

In [1]:
%set_env JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home

env: JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home


In [2]:
import configparser
from datetime import datetime
import os
import util
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [3]:
def create_spark_session(master,endpoint=None):
    spark = SparkSession \
            .builder \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
            .config("fs.s3a.endpoint",endpoint)\
            .appName("udacity-dend-data-lake-proj")\
            .master(master)\
            .getOrCreate()
    
    return spark

In [4]:
#spark://127.0.0.1:7077
spark = create_spark_session("spark://127.0.0.1:7077","s3.us-west-2.amazonaws.com")
spark

# Download files to local directory

In [8]:
base_url = 'https://datasets.imdbws.com'

files_list = [
        "name.basics.tsv.gz",
        "title.akas.tsv.gz",
        "title.basics.tsv.gz",
        "title.crew.tsv.gz",
        "title.episode.tsv.gz",
        "title.principals.tsv.gz",
        "title.ratings.tsv.gz"]
        
temp_filepath = 'file:///Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/tmp'

In [5]:
util.download_files_to_local(base_url, files_list, temp_filepath)

## Process name_basics file

In [6]:
file = 'name.basics.tsv.gz'

df = spark.read.load(temp_filepath+'/'+file,
                     format="csv", sep="\t", inferSchema="true", header="true")

In [7]:
df.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)



In [ ]:
df.describe().show()